In [1]:
import os
import json
import pandas as pd
import numpy as np
from urllib.parse import urlencode
import requests
import codecs
import time
from DataRead import Read

import datetime

In [9]:
read=Read()
kline = read.read_kline('1d','ETHUSDT')
latest_ts=kline.index.max()

latest_ts=str(latest_ts)
latest_ts=int(time.mktime(time.strptime(latest_ts,'%Y-%m-%d %X'))*1000)

In [10]:
kline.index.max()

Timestamp('2022-09-04 08:00:00')

In [11]:
kline.head(2)

,open_time,open,high,low,close,turnover_volume,close_time,turnover_value,deal_num,bid_volume,bid_value,other,symbol
timestamp,,,,,,,,,,,,,
2019-11-18 08:00:00,1574035200000,183.82,184.06,175.01,178.20,293551.23632,1574121599999,5.298573e+07,131823,134230.72271,2.425404e+07,0,ETHUSDT
2019-11-19 08:00:00,1574121600000,178.20,178.52,172.65,175.94,275886.64110,1574207999999,4.841411e+07,125824,138098.32741,2.423493e+07,0,ETHUSDT


In [15]:
kline['EMA12']=kline['close'].ewm(span=12,min_periods=12,adjust=False).mean()
kline['EMA26']=kline['close'].ewm(span=26,min_periods=26,adjust=False).mean()
kline['DIF']=kline['EMA12'] - kline['EMA26']
kline['DEA'] = kline['DIF'].ewm(span=9,min_periods=9,adjust=False).mean()
kline['MACD']=2*(kline['DIF']-kline['DEA'])

In [16]:
kline['MACD'].describe()

count    989.000000
mean      -0.173466
std       68.873466
min     -458.745989
25%      -19.314050
50%        1.949797
75%       25.660376
max      206.294787
Name: MACD, dtype: float64